In [15]:
## Install
!pip install easyocr

In [16]:
## Imports

import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.metrics import roc_auc_score

import re
import string
import nltk
import itertools
import numpy as np
import os
import csv
import io

import torch
import cv2
import easyocr

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# this allows you to apply a progress bar to pandas operations
from tqdm import tqdm
tqdm.pandas()

In [17]:
# easyocr loading
reader = easyocr.Reader(['en']) 

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [18]:
%cd /home/studio-lab-user/DS-Projects/OCR

/home/studio-lab-user/DS-Projects/OCR


In [19]:
# To pull in previusly save data
import pandas as pd
data = pd.read_csv("image_relevant.csv", encoding='latin')
data.head()

,Unnamed: 0,id,description,image,images,relevant,processed_description,text_score
0,0,918707,Boys Among Us Hoodies Super Cool Print Clothes...,968bf5e6c99211c2ad8ac27fa55e4651b941d137,['968bf5e6c99211c2ad8ac27fa55e4651b941d137'],1,boys among us hoodies super cool print clothes...,0.192416
1,1,918708,Cat And Dog Cardboard Printing Jigsaw Puzzle M...,68ece3fab138feb537eee296f3348f3543e23a1d,"['68ece3fab138feb537eee296f3348f3543e23a1d', '...",0,cat dog cardboard printing jigsaw puzzle manuf...,0.030835
2,2,918709,0098 30 Pcs 21pack Sensory 4 Piece Puzzle Fidg...,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,"['a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf', '...",0,0098 30 pcs 21pack sensory 4 piece puzzle fidg...,0.001173
3,3,918710,Boys Sweatshirt Among us 3D Hoodies Cool Fashi...,a278812a832f24a822e2602b91a7b1f5749926cb,['a278812a832f24a822e2602b91a7b1f5749926cb'],1,boys sweatshirt among us 3d hoodies cool fashi...,0.275841
4,4,918711,20cm Among Us Soft Plush Toys Vocal Butt Toy S...,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,['3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a'],0,20cm among us soft plush toys vocal butt toy s...,0.000861


In [20]:
image_df = data[['image','relevant']]

In [21]:
image_df

,image,relevant
0,968bf5e6c99211c2ad8ac27fa55e4651b941d137,1
1,68ece3fab138feb537eee296f3348f3543e23a1d,0
2,a31d8c30452d17221b0d8d4f9ccf50bf8dfff0cf,0
3,a278812a832f24a822e2602b91a7b1f5749926cb,1
4,3f7fc0f877323f4b74cb9a4c3847d3eea6f3ae5a,0
...,...,...
464,43a18980d983e7835ce8a9f50ae8953390197f73,0
465,a2db2c697881750d70feaa1a90d4f271338b9337,0
466,077d6694396e202064fb5b01b42837d2949e4ce7,0
467,a8057438d83d4fc141347237c23e2d3fc714d43f,0


In [22]:
# words and scores
def get_words_and_scores_and_add_to_dataframe(row):
    photo = row["image"]
    if photo == "empty":
        return [[],[]]
    
    img_path = f"/home/studio-lab-user/DS-Projects/OCR/SWOOP_OCR_images/{photo}.jpg"
    # easyocr readtext
    img_text = reader.readtext(img_path)
    #empty lists
    words = []
    confidences = []
    #appending results into the lists
    for (bbox, text, prob) in img_text:
        words.append(text)
        confidences.append(prob)
    result = [words, confidences]
    return result

In [23]:
image_df["words"] = image_df.apply(get_words_and_scores_and_add_to_dataframe, axis=1)
image_df

KeyboardInterrupt: 

In [24]:
import re
import string
def clean_text(row):
    word_list = row["words"][0]
    new_word_list = []
    for word in word_list:
        new_word = word.lower()
        new_word = re.sub("[%s]" % re.escape(string.punctuation), " ", new_word)
        new_word_list.append(new_word)
    return [new_word_list, row["words"][1]]

In [25]:
image_df["words"] = image_df.apply(clean_text, axis=1)

KeyError: 'words'

In [26]:
## define the target words / phrases

targets = ["among us", "impostor"]

In [27]:
# helper functions for funal scoring function

def advance(iterator, step):
    next(itertools.islice(iterator, step, step), None)


def tuplewize(iterable, size):
    iterators = itertools.tee(iterable, size)
    for position, iterator in enumerate(iterators):
        advance(iterator, position)
    return zip(*iterators)

In [28]:
# define function to produce final scores

def get_final_score(row):
    sample_list = row["words"][0]
    score_list = row["words"][1]
    max_score = 0
    
    for target in targets:
        phrase_length = len(target.split())
        for c, s in zip(tuplewize(sample_list, phrase_length), tuplewize(score_list, phrase_length)):
            new_sample = " ".join(c)
            print(new_sample)
            edit_distance = nltk.edit_distance(target, new_sample)
            closeness = (len(target)-edit_distance)/len(target)
            confidence_score = np.mean(s)
            final_score = closeness * confidence_score
            if final_score > max_score:
                max_score = final_score
             
    return max_score

In [29]:
import itertools
import nltk
import numpy as np

image_df["OCR_score"] = image_df.apply(get_final_score, axis=1)

KeyError: 'words'

In [30]:
image_df = image_df.sort_values(by='OCR_score', ascending=False)
pd.set_option('display.max_rows', None)
image_df

KeyError: 'OCR_score'

In [31]:
RA_score = roc_auc_score(image_df["relevant"], image_df["OCR_score"])

print("Rekognition ROC AUC Score =  %.2f" % (RA_score))

KeyError: 'OCR_score'

In [32]:
# Dataframe of true positives

tp = image_df.loc[(image_df['relevant'] == 1) & (image_df['OCR_score'] >= 0.5)] 
print(len(tp))

KeyError: 'OCR_score'

In [33]:
# Dataframe of true negatives

tn = image_df.loc[(image_df['relevant'] == 0) & (image_df['OCR_score'] < 0.5)] 
print(len(tn))

KeyError: 'OCR_score'

In [34]:
# Dataframe of false positives

fp = image_df.loc[(image_df['relevant'] == 0) & (image_df['OCR_score'] >= 0.5)] 
print(len(fp))

KeyError: 'OCR_score'

In [35]:
# Dataframe of false negatives

fn = image_df.loc[(image_df['relevant'] == 1) & (image_df['OCR_score'] < 0.5)] 
print(len(fn))

KeyError: 'OCR_score'

In [36]:
# Precision (What % of irdentified positives and true?)
precision = len(tp) / (len(tp) + len(fp))
print(precision)

NameError: name 'tp' is not defined

In [37]:
# PRecall (% of true positives identified)
recall = len(tp) / (len(tp) + len(fn))
print(recall)

NameError: name 'tp' is not defined

In [38]:
fn

NameError: name 'fn' is not defined

In [39]:
image_df.to_csv('/home/studio-lab-user/DS-Projects/OCR/easy_ocr.csv')